In [1]:
test_input = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""

test_input_mine = """R 12 (#70c710)
U 6 (#0dc571)
L 4 (#5713f0)
D 3 (#d2c081)
L 4 (#59c680)
U 3 (#411b91)
L 4 (#8ceee2)
D 6 (#caa173)"""

In [2]:
input = open("inputs/18").read()

In [3]:
import math

In [4]:
# directions = {
#     "R": (0, 1),
#     "L": (0, -1),
#     "U": (-1, 0),
#     "D": (1, 0)
# }

# lord forgive me im leaving np convention and using math convention
directions = {
    "R": (1, 0),
    "L": (-1, 0),
    "U": (0, 1),
    "D": (0, -1)
}

In [5]:
def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))

def mult_tuple(tpl, c):
    return tuple(c*x for x in tpl)

In [6]:
def p1(input, start=(1, -1)):
    pos = (0, 0)
    dug = [pos]

    for l in input.splitlines():
        dir, num_str, _ = l.split()
        num = int(num_str)

        for i in range(num):
            pos = add_tuples(pos, directions[dir])
            dug.append(pos)

    assert dug[0] == (0,0) and dug[-1] == (0, 0)
    dug = dug[:-1]

    dug_set = set(dug)

    filled = set()

    to_visit = [start]

    while to_visit:
        pos = to_visit.pop()

        # hit edge or already filled
        if pos in dug_set or pos in filled:
            continue
        
        filled.add(pos)

        to_visit.append((add_tuples(pos, directions['D'])))
        to_visit.append((add_tuples(pos, directions['U'])))
        to_visit.append((add_tuples(pos, directions['L'])))
        to_visit.append((add_tuples(pos, directions['R'])))
    
    print(len(filled), len(dug_set))
    
    return len(filled) + len(dug_set)

In [7]:
p1(test_input), p1(input)

24 38
44869 3634


(62, 48503)

In [8]:
p1(test_input_mine, start=(1, 1))

40 42


82

In [9]:
test_hex = "#70c710"

In [10]:
dirs = ['R', 'D', 'L', 'U']

num = int(test_hex[1:-1], base=16)
dir = dirs[int(test_hex[-1])]

dir, num

('R', 461937)

In [11]:
sm = 0
for l in test_input.splitlines():
# for l in input.splitlines():
    _, _, hex_str = l.split()

    dir = dirs[int(hex_str[-2])]
    num = int(hex_str[2:-2], base=16)

    # print(dir, num)
    sm += num

sm

6405262

In [12]:
pos = (0, 0)

horizontal_segments = []
vertical_segments = []

# for l in input.splitlines():
# for l in test_input_mine.splitlines():
for l in test_input.splitlines():
    dir, num_str, _ = l.split()
    num = int(num_str)

    print(dir, num)
    next_pos = add_tuples(pos, mult_tuple(directions[dir], num))

    if dir in ['R', 'L']:
        horizontal_segments.append((pos[1], tuple(sorted((pos[0], next_pos[0])))))
    else:
        vertical_segments.append((pos[0], tuple(sorted((pos[1], next_pos[1])))))

    pos = next_pos

R 6
D 5
L 2
D 2
R 2
D 2
L 5
U 2
L 1
U 2
R 2
U 3
L 2
U 2


In [13]:
horizontal_lines = sorted(list(set([x[0] for x in horizontal_segments])))
vertical_lines = sorted(list(set([x[0] for x in vertical_segments])))

In [14]:
horizontal_lines, vertical_lines

([-9, -7, -5, -2, 0], [0, 1, 2, 4, 6])

In [15]:
import itertools

vertical_sides = []
horizontal_sides = []

vertex_points = []
interior_rectangle = []

for ((x_start, x_end), (y_start, y_end)) in itertools.product(itertools.pairwise(vertical_lines), itertools.pairwise(horizontal_lines)):
    v1 = (x_start, y_start)
    v2 = (x_end, y_start)

    v3 = (x_start, y_end)
    v4 = (x_end, y_end)

    vertex_points.extend([v1, v2, v3, v4])
    x_extent = x_end - x_start > 1
    y_extent = y_end - y_start > 1

    # keep track of all four sides, but only storing the interior of the sides
    # use the same format for horizontal and vertical sides that I did above

    if x_extent:
        horizontal_sides.append((y_start, (x_start+1, x_end-1)))
        horizontal_sides.append((y_end, (x_start+1, x_end-1)))

    if y_extent:
        vertical_sides.append((x_start, (y_start+1, y_end-1)))
        vertical_sides.append((x_end, (y_start+1, y_end-1)))

    # now keep the interior of the rectangle
    if x_extent and y_extent:
        interior_rectangle.append((x_start+1, x_end-1, y_start+1, y_end-1))

In [16]:
vertical_sides = set(sorted(list(set(vertical_sides))))
horizontal_sides = set(sorted(list(set(horizontal_sides))))
vertex_points = sorted(list(set(vertex_points)))

In [17]:
def oned_intersection(interval1, interval2):
    # boundaries inclusive
    # just checks if two intervals intersect, returns bool
    if interval1[0] > interval2[0]:
        interval1, interval2 = interval2, interval1
    
    return interval1[1] >= interval2[0]

In [39]:
def point_on_boundary(p):
    for h in horizontal_segments:
        if p[1] == h[0] and oned_intersection(h[1], (p[0], p[0])):
            return True
    
    for v in vertical_segments:
        if p[0] == v[0] and oned_intersection(v[1], (p[1], p[1])):
            return True
    
    return False

In [44]:
def point_in_interior(p, check_vertical=True):
    x, y = p

    if check_vertical:
        lines_less = [l for l in vertical_lines if l < x]
        on_boundary = [point_on_boundary((l, y)) for l in lines_less]
    else:
        lines_less = [l for l in horizontal_lines if l < y]
        on_boundary = [point_on_boundary((x, l)) for l in lines_less]

    # count how many transitions occurred, grouping adjacent True values
    n = sum(int(b) for b, _ in itertools.groupby(on_boundary))

    return n % 2 == 1

In [24]:
total_rectangle_interior_area = 0
for (x_start, x_end, y_start, y_end) in interior_rectangle:
    # just check if point from rectangle is interior
    if point_in_interior((x_start, y_start)):
        ia = (1+x_end - x_start) * (1+y_end - y_start)
        total_rectangle_interior_area += ia
    else:
        pass

[(1, (-8, -8)), (6, (-8, -8))]
[(0, (-6, -6)), (4, (-6, -6))]
[(2, (-4, -3)), (6, (-4, -3))]
[(0, (-1, -1)), (6, (-1, -1))]
[(1, (-8, -8)), (6, (-8, -8))]
[(0, (-6, -6)), (4, (-6, -6))]
[(2, (-4, -3)), (6, (-4, -3))]
[(0, (-1, -1)), (6, (-1, -1))]


In [25]:
total_rectangle_interior_area

9

In [52]:
zero_d_area = 0

for p in vertex_points:
    if point_on_boundary(p):
        zero_d_area += 1
    else:
        r1 = point_in_interior(p, check_vertical=True)
        r2 = point_in_interior(p, check_vertical=False)

        assert r1 == r2

        if r1:
            zero_d_area += 1

In [53]:
zero_d_area

24

In [56]:
one_d_area = 0

for (x, (y1, y2)) in vertical_sides:
    # just take a single point from each side
    p = (x, y1)
    l = y2 - y1 + 1

    if point_on_boundary(p):
        one_d_area += l
    else:
        print(p)
        r1 = point_in_interior(p, check_vertical=True)
        r2 = point_in_interior(p, check_vertical=False)

        assert r1 == r2

        if r1:
            one_d_area += l

(0, -4)


AssertionError: 